<h1><center>Final Year Project 2023 <br> Video Summarization using Memorability Scores</center></h1>

<h1><center>
Video Memorability Prediction<br>
Ramya priya S<br>
Thushita Mariaselvan
</center></h1>

## Google Colab Setup

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive/')

Mounted at /content/drive/


## Import and Installing packages

Few main pakages used in this project are as follows:

> * **pyprind:** The PyPrind (Python Progress Indicator) module provides a progress bar and a percentage indicator object that let you track the progress of a loop structure or other iterative computation.
> * **Keras:** is an open-source neural-network library written in Python. It is capable of running on top of TensorFlow. Designed to enable fast experimentation with deep neural networks.
> * **TensorFlow:** is a free and open-source software library for dataflow and differentiable programming across a range of tasks. It is a symbolic math library, and is also used for machine learning applications such as neural networks.
> * **Scikit-learn:** is a machine learning library for the Python. It features various classification, regression and clustering algorithms including support vector machines, random forests.



In [ ]:
!pip install pyprind
!pip install -U keras
!pip install keras
!pip install tensorflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import keras
from keras import Sequential, layers, regularizers, optimizers
import numpy as np
from string import punctuation
import pyprind
from collections import Counter
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy import stats
from natsort import natsorted,ns
import os

## Setting seed for reproducibility

By doing so our program generates the same set of random inputs to help us reproduce the results later

In [ ]:
import tensorflow as tf
tf.random.set_seed(1234)

## Spearman's Correlation Function To Calculate Memorability Score

>  **Spearman's rank correlation coefficient :** is a nonparametric measure of rank correlation (statistical dependence between the rankings of two variables). It assesses how well the relationship between two variables can be described using a linear function. We have calculated the spearman score between the predicted and actual values. 

In [ ]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        #if it has only long / short term calc
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
          #if it has both long ans short term calc for both seperately and print
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

## Features

---
We are going to load 3 features from the dataset.

*   Convolution 3D (C3D) -- Generic features generated for video analysis.
> C3D is obtained by training a deep 3D convolutional network on a large annotated video dataset. The dataset contains various concepts encompassing objects, actions, scenes and other frequently occurring categories in videos.

*   Histogram of Motion Patterens (HMP) -- Histogram of motion patterns generated for each video
> HMP is a static image template which helps in understanding the motion location and path as it progresses. The temporal motion information is collapsed into a single image template where intensity is a function of recency of motion, where brighter values correspond to a more recent motion.

*  Captions -- Textual features describing the videos.
> Captions provide a textual description of an video based on the objects and actions in it.





### Functions for Loading different Features

In [ ]:
# captions
def read_caps(fname):
    """Load the captions into a dataframe"""
    vn = []
    cap = []
    df = pd.DataFrame();
    with open(fname) as f:
        for line in f:
            pairs = line.split()
            vn.append(pairs[0])
            cap.append(pairs[1])
        df['video']=vn
        df['caption']=cap
    return df
 
# loading C3D
def read_C3D(fname):
    with open(fname) as f:
        for line in f:
            C3D =[float(item) for item in line.split()] 
    return C3D
  
# loading HMP
def read_HMP(fname):
    """Scan HMP(Histogram of Motion Patterns) features from file"""
    #It is in form of bin number : values
    #Hence getting these and storing it in dict in {bin no:value} form 
    #then storing it in list with index as bin no
    with open(fname) as f:
        for line in f:
            pairs=line.split()
            HMP_temp = { int(p.split(':')[0]) : float(p.split(':')[1]) for p in pairs}
    # there are 6075 bins, fill zeros
    HMP = np.zeros(6075)
    for idx in HMP_temp.keys():
        HMP[idx-1] = HMP_temp[idx]            
    return HMP

## Declaring Source Paths And Variables
> Ground Truth values are loaded as labels and contains:
*   Video's name
*   Short-term memorability score.
*   Number of annotations which was used to calculate its short-term memorability score.
*   Long-term memorability score.
*   Number of annotations which was used to calculate its long-term memorability score.








In [ ]:
# load the captions
cap_path = '/content/drive/MyDrive/dev-set_video-captions.txt'
df_cap=read_caps(cap_path)

# load the ground truth values
label_path = '/content/drive/MyDrive/ground-truth_dev-set.csv'
labels=pd.read_csv(label_path)

# Path for the features
Feat_path = '/content/drive/MyDrive/'

#getting video names using features' name
def getnames(featurename):
  vnames = os.listdir(Feat_path+featurename)
  return vnames

#get full path of the features using features' name 
def getpaths(featurename):
  fpath = [Feat_path+featurename+'/' + x for x in  getnames(featurename)]
  sorted = natsorted(fpath,alg=ns.IGNORECASE)
  print('Feature Path:')
  print(sorted[:5])
  return sorted


### Functions Definitions
> For reading HMP and C3D text files into an numpy array



In [ ]:
#load C3D and return as an array of list
def df_C3D():
  path = getpaths('C3D')
  c3d = []
  print('Loading C3D')
  for item in path:
    c3d.append(read_C3D(item))
  print('done')
  return np.asarray(c3d)

#load HMP and return as an array of list
def df_HMP():
  path = getpaths('HMP')
  hmp = []
  print('Loading HMP')
  for item in path:
    if 'video' not in item:
      continue
    hmp.append(read_HMP(item))
  print('done')
  return np.asarray(hmp)

### Loading C3D and HMP features.

In [ ]:
#loading C3D
feature1 = df_C3D()

Feature Path:
['/content/drive/MyDrive/C3D/0n3gVXmbG-I_part_0.txt', '/content/drive/MyDrive/C3D/0n3gVXmbG-I_part_1.txt', '/content/drive/MyDrive/C3D/0n3gVXmbG-I_part_2.txt', '/content/drive/MyDrive/C3D/0n3gVXmbG-I_part_3.txt', '/content/drive/MyDrive/C3D/3-sFLOcz_1I_part_0.txt']
Loading C3D
done


In [ ]:
feature2 = df_HMP()

Feature Path:
['/content/drive/MyDrive/HMP/0n3gVXmbG-I_part_0.txt', '/content/drive/MyDrive/HMP/0n3gVXmbG-I_part_1.txt', '/content/drive/MyDrive/HMP/0n3gVXmbG-I_part_2.txt', '/content/drive/MyDrive/HMP/0n3gVXmbG-I_part_3.txt', '/content/drive/MyDrive/HMP/0n3gVXmbG-I_part_4.txt']
Loading HMP
done


## Cleaning / Preprocessing of features

### Optimizing captions

Natural language toolkit library has be used to optimize the captions.

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re

> Stripping Special Characters

In [ ]:
df = df_cap.copy()
import re
def strip_character(dataCol):
    r = re.compile(r'[^a-zA-Z]')
    #substituting special characters with space
    return r.sub(' ', str(dataCol))

df['caption'] = df['caption'].apply(strip_character)

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True



> Removing Stopwords
######Stop word removal is one of the most commonly used preprocessing steps across different NLP applications. The idea is simply removing the words that occur commonly across all the documents in the corpus. Typically, articles and pronouns are generally classified as stop words.



In [ ]:
stop = stopwords.words('english') 

In [ ]:
df['caption'] = df['caption'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))
df['caption'].head()

0            couple relaxing picnic crane shot
1    cute black white cats cage animal shelter
2                               owl tree close
3                          around house laptop
4                   beautiful young girl apple
Name: caption, dtype: object



> Lemmatization
*   Lemmatisation is a process of grouping words together which inflict same forms of a word so they can be analysed as a single item.
* For example, reducing "builds", "building", or "built" to the lemma "build"

In [ ]:
import nltk
nltk.download('omw-1.4')
df['caption'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]',' ',text)) for text in lis]) for lis in df['caption']]
df['caption'].head()

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


0            couple relaxing picnic crane shot
1    cute black white cats cage animal shelter
2                               owl tree close
3                          around house laptop
4                   beautiful young girl apple
Name: caption, dtype: object



> Extracting Unique Word Count



In [ ]:
counts = Counter()
# pbar = pyprind.ProgBar(len(df['caption']), title='Counting word occurrences')
for i, cap in enumerate(df['caption']):
    counts.update(cap.split())   

In [ ]:
df.caption.values

array(['couple relaxing picnic crane shot',
       'cute black white cats cage animal shelter', 'owl tree close', ...,
       'tombstone hattie wife private charles thompson',
       'closeup green vegetation desert field',
       'fisherman step shore corpus christi texas'], dtype=object)



> n-Gram approach using TF - IDF




In [ ]:
vect = TfidfVectorizer(ngram_range = (1,4)).fit(df.caption)
vect_transformed_X_train = vect.transform(df.caption)
len_token = len(vect.get_feature_names_out())

In [ ]:
len_token

74842



>  Maping each unique word in the vector to an integer (one-hot encoding)


*   Building word index by using tokenizer
*   Fitting a list of captions to the tokinizer
> tokenizer vectorizes a text corpus, by turning each text into a sequence of integers





In [ ]:
len_token = len(counts)
tokenizer = Tokenizer(num_words=len_token)

In [ ]:
tokenizer.fit_on_texts(list(vect.get_feature_names_out()))

In [ ]:
captions = tokenizer.texts_to_matrix(list(df.caption.values),mode='binary')

In [ ]:
len(captions)

8000

In [ ]:
#Checking length. Output should be (8000,8000)
feature1 = feature1[:8000]
print(len(feature1),len(feature2))

8000 8000


###Testing data captions preprocessing

In [ ]:

bday = ['young girl sitting in a table talks with plate food'
,'some children look into a bag and take out something'
,'woman comes out of bathroom downstairs laughing'
,'women is driving in car and talking'
,'a boy is riding a bike around the room with a girl'
,'a girl wearing a bag is standing on the stairs and talking'
,'a girl blowing baloon boy pumping up them'
,'a girl and boy looks at phone takes it'
,'the children play with balloons hitting it'
,'two girls sitting in front of cake and put candle on cake'
,'girl closes eyes opens blows candle on cake'
,'girl and women talk to the camera']

bday_df = pd.DataFrame(list(zip(bday)),
              columns=['caption'])

#stopwords removal
bday_df['caption'] = bday_df['caption'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))
#lemmatization
bday_df['caption'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]',' ',text)) for text in lis]) for lis in bday_df['caption']]
print(bday_df['caption'].head())

captions_bday = tokenizer.texts_to_matrix(list(bday_df.caption.values),mode='binary')
print(captions_bday[0])
       

0    young girl sitting table talks plate food
1             children look bag take something
2     woman comes bathroom downstairs laughing
3                    women driving car talking
4             boy riding bike around room girl
Name: caption, dtype: object
[0. 0. 0. ... 0. 0. 0.]


## Model Implementations
>Models that we have chosen to process futher are:
*    Linear Regression
*    Support Vector Regression (SVR)
*    Random Forest Regression
*    Ridge Regression
*    Ensemble approach using SVR, Decision Trees, Linear Regression as base estimators and Ridge Regression as final estimator



### Regression Techniques


> As we have seen in the Neural Network section we can choose the feature we need and load it into out 'X' Variabel
*    To reduce the code we will be exhibiting the model with best results among the features 



#### Linear Regression

In [ ]:
#Short term and long term memorability score 
Y_LR = labels[['short-term_memorability','long-term_memorability']].values
#c3d values
X_LR = feature1

In [ ]:
#Splitting dataset into test and train set with 90:10 ratio
X_train, X_test, Y_train, Y_test = train_test_split(X_LR,Y_LR, test_size=0.20, random_state=42)

In [ ]:
# !pip install ​​​scikit-optimize==0.8.1
# !pip install scikit-learn==0.22.2
from sklearn.linear_model import LinearRegression

#Creating A Linear Regression Model
lr = LinearRegression()

#Training the model
history = lr.fit(X_train,Y_train)

In [ ]:
predictionsLR = lr.predict(X_test)
print(predictionsLR.shape)

(1600, 2)


#### Support Vector Regression -- SVR
Finds the best fit line(hyperplane) that has the maximum number of points

###### Short Term Memorability

In [ ]:
from sklearn.svm import SVR

In [ ]:
Y_short = labels[['short-term_memorability']].values
X_SVR = np.concatenate((feature1,feature2,captions), axis=1)

In [ ]:
#Splitting dataset into test and train set with 90:10 ratio
X_train, X_test, Y_train_short, Y_test_short = train_test_split(X_SVR,Y_short, test_size=0.1, random_state=42)

In [ ]:
print(len(X_SVR),Y_short.shape)

8000 (8000, 1)


In [ ]:
#Creating an SVR model
#The C parameter tells the SVM optimization how much you want to avoid misclassifying each training example.
modelSVR_short = SVR(C=100).fit(X_train,Y_train_short)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
predictionsSVR_short = modelSVR_short.predict(X_test)

In [ ]:
Get_score(predictionsSVR_short,Y_test_short)

The Spearman's correlation coefficient is: 0.415


###### Long Term Memorability

In [ ]:
from sklearn.svm import SVR

In [ ]:
Y_long = labels[['long-term_memorability']].values
X_SVR = np.concatenate((feature1,captions,feature2), axis=1)

In [ ]:
#Splitting dataset into test and train set with 90:10 ratio
X_train, X_test, Y_train_long, Y_test_long = train_test_split(X_SVR,Y_long, test_size=0.1, random_state=42)

In [ ]:
print(len(X_SVR),Y_short.shape)

8000 (8000, 1)


In [ ]:
modelSVR_long = SVR(C=100).fit(X_train,Y_train_long)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
predictionsSVR_long = modelSVR_long.predict(X_test)

In [ ]:
Get_score(predictionsSVR_long,Y_test_long)

The Spearman's correlation coefficient is: 0.170


#### Ridge Regression

In [ ]:
Y_NN = labels[['short-term_memorability','long-term_memorability']].values
X_NN =np.concatenate((captions,feature1,feature2), axis=1)

In [ ]:
#Splitting dataset into test and train set with 90:10 ratio
X_train, X_test, Y_train, Y_test = train_test_split(X_NN,Y_NN, test_size=0.10, random_state=42)

In [ ]:
print(X_train.shape,Y_train.shape)

(7200, 11953) (7200, 2)


In [ ]:
from sklearn.linear_model import Ridge
clf = Ridge()

#Creating an Ridge Regression Model
clf.set_params(alpha=45)
model = clf.fit(X_train, Y_train) 

In [ ]:
predictionsRR = model.predict(X_test)

In [ ]:
predictionsRR.shape

(800, 2)

In [ ]:
Get_score(predictionsRR, Y_test)

The Spearman's correlation coefficient is: 0.454
The Spearman's correlation coefficient is: 0.185


#### Random Forest
 A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#Short term and long term memorability score 
Y_RF = labels[['short-term_memorability','long-term_memorability']].values
X_RF = np.concatenate((feature1,feature2,captions), axis=1)

In [ ]:
#Splitting dataset into test and train set with 90:10 ratio
X_train, X_test, Y_train, Y_test = train_test_split(X_RF,Y_RF, test_size=0.10, random_state=42)

#setting parameters for Random Forest
# n_estimators = number of trees max_depth = maximium depth of the tree
rf = RandomForestRegressor(n_estimators=20,max_depth=10,random_state=50,verbose=2)

#training dataset
rf.fit(X_train,Y_train)

print(rf.decision_path(X_train))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 20


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.8s remaining:    0.0s


building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  6.1min finished


(<7200x8186 sparse matrix of type '<class 'numpy.int64'>'
	with 1576481 stored elements in Compressed Sparse Row format>, array([   0,  305,  800, 1467, 1742, 2251, 2694, 3095, 3334, 3755, 4202,
       4651, 4910, 5259, 5628, 6063, 6466, 6939, 7470, 7835, 8186]))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


In [ ]:
predictionsRF = rf.predict(X_test)
print(predictionsRF.shape)

(800, 2)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


In [ ]:
Get_score(predictionsRF,Y_test)

The Spearman's correlation coefficient is: 0.223
The Spearman's correlation coefficient is: 0.120


### Stacked Regression - Ensemble approach
>Stacking, an ensemble machine learning algorithm that learns how to best combine each of the models in an ensemble to come up with the best performance.

>Level-0 Models (Base-Models): Models fit on the training data and whose predictions are compiled.
Level-1 Model (Meta-Model): Model that learns how to best combine the predictions of the base models.

> Ensemble approach using
SVR, Decision Tree’s and Linear Regression as base
classifiers and Ridge Linear Regression as the final
estimator.

#### Short Term Memorability Score

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

In [ ]:

import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.linear_model import Ridge

#Short term and long term memorability score 
Y_Stack = labels[['short-term_memorability']].values
X_Stack = captions

In [ ]:
#Splitting dataset into test and train set with 90 : 10 ratio
X_train, X_test, Y_train, Y_test = train_test_split(X_Stack,Y_Stack, test_size=0.10, random_state=42)

In [ ]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

estimators = [
    ('regr', DecisionTreeRegressor(max_depth=10)),
    ('svr', SVR(C=100)),
    ('Lr',LinearRegression())
]
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(alpha=45)
)

model = reg.fit(X_train,Y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.62505e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [ ]:
StackPredictions = model.predict(X_test)

In [ ]:
X_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [ ]:
X_test.shape,Y_test.shape,StackPredictions.shape

((800, 5777), (800, 1), (800,))

In [ ]:
print("Short_Term Memorability")
for i in range(10):
  print(StackPredictions[i])


Short_Term Memorability
0.8455822336120644
0.8616276299006287
0.8607597382831513
0.8616069728065945
0.8609540891754777
0.8624627006631178
0.8510350233753313
0.8613135752630425
0.8638120459583756
0.853517146105826


In [ ]:
print("Difference between the actual and predicited values")
for i in range(10):
  print(abs(StackPredictions[i]-Y_test[i]))

Difference between the actual and predicited values
[0.04941777]
[0.05837237]
[0.00224026]
[0.06339303]
[0.05095409]
[0.0354627]
[0.06503502]
[0.11968642]
[0.03418795]
[0.13351715]


In [ ]:
Get_score(StackPredictions, Y_test)

The Spearman's correlation coefficient is: 0.440


In [ ]:
Predictions_bday = model.predict(captions_bday)
for i in range(len(Predictions_bday)):
  if(Predictions_bday[i]>0.85):
    print(i,bday[i],Predictions_bday[i])
  else:
    print('low')
    print(bday[i],Predictions_bday[i])

test_mem_score = pd.DataFrame(list(zip(bday, Predictions_bday)),
              columns=['captions','mem_scores'])

# test_mem_score.sort_values('mem_scores')

0 young girl sitting in a table talks with plate food 0.8696345522053622
1 some children look into a bag and take out something 0.8586100826193283
2 woman comes out of bathroom downstairs laughing 0.8656985789794267
3 women is driving in car and talking 0.8628422948175682
4 a boy is riding a bike around the room with a girl 0.8672386743630695
5 a girl wearing a bag is standing on the stairs and talking 0.8675293532849079
6 a girl blowing baloon boy pumping up them 0.8670025074512311
7 a girl and boy looks at phone takes it 0.8674027969114809
8 the children play with balloons hitting it 0.8556852995215515
9 two girls sitting in front of cake and put candle on cake 0.863903695511688
10 girl closes eyes opens blows candle on cake 0.8657154137385624
11 girl and women talk to the camera 0.8680296160879852


#### Long Term Memorability Score

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

In [ ]:
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict

#long term memorability score 
Y_Stack = labels[['long-term_memorability']].values
X_Stack = np.concatenate((captions,feature2), axis=1)

In [ ]:
#Splitting dataset into test and train set with 90:10 ratio
X_train, X_test, Y_train, Y_test = train_test_split(X_Stack,Y_Stack, test_size=0.10, random_state=42)

In [ ]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

estimators = [
    ('regr', DecisionTreeRegressor(max_depth=10)),
    ('svr', SVR(C=100)),
    ('Lr',LinearRegression())
]
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(alpha=45)
)

model = reg.fit(X_train,Y_train)

In [ ]:
StackPredictions = model.predict(X_test)

In [ ]:
X_test.shape,Y_test.shape,StackPredictions.shape

In [ ]:
print("Long_Term Memorability")
for i in range(10):
  print(StackPredictions[i])


In [ ]:
print("Difference between the actual and predicited values")
for i in range(10):
  print(abs(StackPredictions[i]-Y_test[i]))

In [ ]:
Get_score(StackPredictions, Y_test)

# References

* https://pypi.org/project/PyPrind/
* https://en.wikipedia.org/wiki/Keras
* https://www.tensorflow.org/
* https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient
* https://github.com/dazcona/memorability

